# 环境配置

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from tqdm import trange

# The multivariate Variance Gamma model basket option pricing and calibration 论文复刻

Parameter  
$r = 3\%$  
$\rho_{i,j}=0, ~for~i \ne j ~and~ i, j = 1, 2, 3$  
$N = 10^6$

| 1| 1 | 2 | 3|
|- |- |- |- |
|$\mu$|-0.1368 | -0.056 | -0.1984 |
|$\sigma$| 0.1099 | 0.1677 | 0.0365 |
| $X(0)$ | 100 | 200 | 300 |
| $\omega$ | 1/3 | 1/6 | 1/9 |

$$\varepsilon[K]=\frac{|\overline{C}[K] - C^{sim}[K]|}{C^{sim}[K]}$$

In [2]:
r = 0.03
rho = np.eye(3)

## table1

In [3]:
X_0 = np.array([100, 200, 300])
q = np.array([0, 0, 0])
omega = np.array([0, 0, 0])
mu = np.array([-0.1368, -0.056, -0.1984])
sigma = np.array([0.1099, 0.1677, 0.0365])
weight = np.array([1/3, 1/6, 1/9]) # w是权重，omega才是参数
K = np.array([0, 0, 0])
parameter_list = [X_0, q, omega, mu, sigma, weight, K]

table1 = pd.DataFrame(data =  {'X_0': X_0, 'mu': mu,'sigma': sigma, 'weight' : weight},
                      index =  ['stock 1', 'stock 2', 'stock 3'])
table1

,X_0,mu,sigma,weight
stock 1,100,-0.1368,0.1099,0.333333
stock 2,200,-0.0560,0.1677,0.166667
stock 3,300,-0.1984,0.0365,0.111111


## table2

先做个表头

In [4]:
from itertools import product
# T, v, K, C_hat, C_sim, vare

T = [1, 2]
v = [0.5, 0.75, 0.9]
K = [75, 90, 100, 110, 125]

table2 = pd.DataFrame(list(product(T, v, K)), columns = ['T', 'v', 'K']).set_index(['T','v', 'K'])
table2.loc[:,['$\overline{C}[K]$','$C^sim[K]$', r'$\varepsilon[K]$']] = 0
table2.T

T                    1                                               ...    2  \
v                 0.50                     0.75                      ... 0.75   
K                  75   90   100  110  125  75   90   100  110  125  ...  75    
$\overline{C}[K]$  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
$C^sim[K]$         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
$\varepsilon[K]$   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

T                                                               
v                                     0.90                      
K                  90   100  110  125  75   90   100  110  125  
$\overline{C}[K]$  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
$C^sim[K]$         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
$\varepsilon[K]$   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[3 rows x 30 columns]

### 计算所有T, v, K

In [5]:
from scipy.stats import norm
import scipy.integrate as sci
import math
from scipy.stats import norm
from scipy.optimize import fsolve # 解决非线性方程组的数值求解问题
from scipy.optimize import bisect # 解决非线性方程组的数值求解问题
from scipy.stats import gamma # gamma分布的概率密度

#### upper

In [6]:
from tqdm import tqdm
C_upper_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
    K_i = K / weight.copy() / len(table1)
    def f(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        d_1 = (np.log(X_0/K_i) + (r - q + omega) * T + mu * x + sigma ** 2 * x)/(sigma * np.sqrt(x))
        d_2 = d_1 - sigma * np.sqrt(x)
        exp_upper = np.exp((r - q + omega ) * T + (mu + sigma ** 2 / 2) * x)
        X_price_numerical = ((X_0 * exp_upper * norm.cdf(d_1) - K_i * norm.cdf(d_2)))
        S_price_numerical = np.dot(X_price_numerical, weight)
        return S_price_numerical * gamma_density * np.exp(-r * T)
    C_upper = sci.quad(f, 0, np.inf)[0]
    C_upper_lst.append(C_upper)

100%|██████████| 30/30 [00:04<00:00,  6.24it/s]


In [7]:
table2.loc[:,'$C^u$'] = C_upper_lst
# table2

#### lower

$$\lambda_i = w_j X_j(0)\exp\{(r-q_j+\omega_j)T+\mu_j y+\frac{\sigma_j^2 y}{2}\}$$  
$$\sigma_{\Lambda_y}^2=\sum_{i=1}^n\lambda_i^2\sigma_i^2+\sum_{i=1, i\ne j}^n \lambda_i \lambda_j \sigma_i \sigma_j \rho_{i,j}$$  
$$r_i=\frac{\sum_{i=1}^n\lambda_j\sigma_j\rho_{i,j}}{\sigma_{\Lambda_y}}$$

In [8]:
from tqdm import tqdm
C_lower_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    def g(x):
        omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
        K_i = K / weight.copy() / len(table1)
        lambda_i = weight * X_0 * np.exp((r - q + omega) * T + (mu + sigma ** 2 / 2) * x)
        sigma2Lambda = np.dot(np.dot((lambda_i * sigma),rho),(lambda_i * sigma)) # 是个矩阵
        r_i = np.dot((lambda_i * sigma),rho)/sigma2Lambda
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        d_1 = (np.log(X_0/K_i) + (r - q + omega) * T + mu * x + sigma ** 2 * x / 2 * (1 + r_i ** 2))/(sigma * np.sqrt(x) * r_i)
        d_2 = d_1 - sigma * np.sqrt(x) * r_i
        exp_lower = np.exp((r + omega - q) * T + (mu + sigma ** 2 / 2) * x)
        X_price_numerical = ((X_0 * exp_lower * norm.cdf(d_1) - K_i * norm.cdf(d_2)))
        S_price_numerical = np.dot(X_price_numerical, weight)
        return S_price_numerical * gamma_density * np.exp(-r * T)
    C_lower = sci.quad(g, 0, np.inf)[0]
    C_lower_lst.append(C_lower)

100%|██████████| 30/30 [00:08<00:00,  3.64it/s]


In [9]:
table2.loc[:,'$C^l$'] = C_lower_lst
table2 = table2.round(4)
# table2

#### z_y to approximate

##### var upper

\begin{align}
& \operatorname{Var}\left[S_y^c\right]=\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) \\
& \quad e^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j\right) y+\frac{\sigma_i^2+\sigma_j^2}{2} y}\left(e^{\sigma_i \sigma_j y}-1\right) .
\end{align}

In [10]:
from tqdm import tqdm
var_upper_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    def var_upper_calculate(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_upper_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_upper_part2 = np.exp(2 * r * T)
        var_upper_part3 = np.outer(np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x),np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x))
        var_upper_part4 = np.exp(np.outer(sigma,sigma) * x) - 1
        var_upper_int = sum(sum(var_upper_part1 * var_upper_part2 * var_upper_part3 * var_upper_part4))
        return var_upper_int * gamma_density
    var_upper = sci.quad(var_upper_calculate, 0, np.inf)[0]
    var_upper_lst.append(var_upper)

100%|██████████| 30/30 [00:02<00:00, 10.64it/s]


In [11]:
table2.loc[:,'$Var^u$'] = var_upper_lst
# table2

\begin{align*}
\operatorname{Var}  {\left[S_y^l\right]}&={\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) } \\
&\quad \times \mathrm{e}^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j+\frac{1}{2}\left(\sigma_i^2\left(1-r_i^2\right)+\sigma_j^2\left(1-r_j^2\right)\right)\right) y} \\
&\quad \times e^{\frac{1}{2}(r^2_i\sigma^2_i+r^2_j\sigma^2_j)y}(e^{r_ir_j\sigma_i\sigma_jy}-1)\\
& {=\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) } \\
&\quad \times \mathrm{e}^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j+\frac{1}{2}\left(\sigma_i^2+\sigma_j^2\right)\right) y} \\
& \quad\times (e^{r_ir_j\sigma_i\sigma_jy}-1)
\end{align*}

权重是$w$  
$$\lambda_i = w_j X_j(0)\exp\{(r-q_j+\omega_j)T+\mu_j y+\frac{\sigma_j^2 y}{2}\}$$  
$$\sigma_{\Lambda_y}^2=\sum_{i=1}^n\lambda_i^2\sigma_i^2+\sum_{i=1, i\ne j}^n \lambda_i \lambda_j \sigma_i \sigma_j \rho_{i,j}$$  
$$r_i=\frac{\sum_{i=1}^n\lambda_j\sigma_j\rho_{i,j}}{\sigma_{\Lambda_y}}$$

##### var lower 

In [12]:
from tqdm import tqdm
var_lower_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    def var_lower_calculation(x):
        lambda_i = weight * X_0 * np.exp((r - q + omega) * T + (mu + sigma ** 2 / 2) * x)
        sigma2Lambda = np.dot(np.dot((lambda_i * sigma),rho),(lambda_i * sigma)) # 是个矩阵
        r_i = np.dot((lambda_i * sigma),rho)/np.sqrt(sigma2Lambda)
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_lower_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_lower_part2 = np.exp(2 * r * T)
        var_lower_part3 = np.outer(np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x),np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x))
        var_lower_part4 = np.exp(np.outer(sigma * r_i, sigma * r_i) * x) - 1
        var_lower_int = sum(sum(var_lower_part1 * var_lower_part2 * var_lower_part3 * var_lower_part4))
        return var_lower_int * gamma_density
    var_lower = sci.quad(var_lower_calculation, 0, np.inf)[0]
    var_lower_lst.append(var_lower)

100%|██████████| 30/30 [00:02<00:00, 10.12it/s]


In [13]:
table2.loc[:,'$Var^l$'] = var_lower_lst
# table2

##### var Sy

\begin{align*}
	\mathrm{Var}[{_y}S]=\sum_{i=1}^{n} \sum_{j=1}^{n} w_{i} w_{j} X_{i}(0) X_{j}(0)  
	\mathrm{e}^{2 r T+\left(\omega_{i}-q_{i}+\omega_{j}-q_{j}\right) T+\left(\mu_{i}+\mu_{j}\right) y} 
	e^{\frac{\sigma_i^2+\sigma_j^2+\rho_{i,j}\sigma_i\sigma_j
	}{4-\rho^2_{i,j}}2(1-\rho^2_{i,j})y}
\end{align*}


In [14]:
from tqdm import tqdm
var_Sy_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    def var_calculation(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_part2 = np.exp(2 * r * T)
        var_part3 = np.outer(np.exp((weight - q) * T + mu * x),np.exp((weight - q) * T + mu * x))
        var_part4_i_ne_j = np.exp((sigma ** 2 + (sigma ** 2).reshape(len(sigma),-1) + np.outer(sigma, sigma) * rho) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))
        np.fill_diagonal(var_part4_i_ne_j, np.outer(np.exp(x * sigma ** 2), np.exp(x * sigma ** 2 * 2))) # 生成E[XY] # TODO
        var_part4 = var_part4_i_ne_j - np.outer(np.exp(sigma ** 2 / 2 * x), np.exp(sigma ** 2 / 2 * x))
        # var_part4 = (np.exp((sigma ** 2) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))) * (np.exp((sigma ** 2) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))) #* np.exp(np.outer(sigma,sigma) * rho  * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))
        var_int = sum(sum(var_part1 * var_part2 * var_part3 * var_part4))
        return var_int * gamma_density
    var_Sy = sci.quad(var_calculation, 0, np.inf)[0]
    var_Sy_lst.append(var_Sy)

100%|██████████| 30/30 [00:02<00:00, 10.88it/s]


In [15]:
table2.loc[:,'$Var[Sy]$'] = var_Sy_lst
# table2

$$z_y = \frac{Var[S_y^c]-Var[S_y]}{Var[S_y^c]-Var[S_y^l]}$$  

In [16]:
table2.loc[:,'$z_y$'] = (table2.loc[:,'$Var^u$'] - table2.loc[:,'$Var[Sy]$'])/(table2.loc[:,'$Var^u$'] - table2.loc[:,'$Var^l$'] )
table2.loc[:,'$\overline{C}[K]$'] = table2.loc[:,'$z_y$'] * table2.loc[:,'$C^l$'] + (1-table2.loc[:,'$z_y$']) * table2.loc[:,'$C^u$']
# table2

#### sim

##### 生成随机数

##### 蒙特卡洛（向量化，指令并行）

In [18]:
from tqdm import tqdm
C_sim_list = list()
C_sim_result_list= list()
power_num = 6
sim_times = 10 ** power_num
for (T,v,K) in tqdm(table2.index.values):
    np.random.seed(20240321) 
    Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=rho, size=sim_times)
    Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)
    omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
    K_i = K / weight.copy() / len(table1)
    X_price_vector = X_0 * np.exp((r - q + omega) * T) * np.exp(np.outer(mu, Gamma_values)).T * np.exp(sigma * np.sqrt(Gamma_values.reshape(-1, 1)) * Normal_values)
    S_price_vector = np.dot(X_price_vector, weight)
    C_sim_vector_list = (S_price_vector - K) * (S_price_vector - K > 0)
    C_sim = sum(C_sim_vector_list) * np.exp(-r * T) / len(C_sim_vector_list) # 原论文是27.3230
    C_sim_result_list.append(C_sim)
    C_sim_list = list()

100%|██████████| 30/30 [00:09<00:00,  3.14it/s]


In [19]:
table2.loc[:,'$C^sim[K]$'] = C_sim_result_list
table2.loc[:,r'$\varepsilon[K]$'] = abs(table2.loc[:,'$\overline{C}[K]$'] - table2.loc[:,'$C^sim[K]$'])/ table2.loc[:,'$C^sim[K]$']
# table2

In [20]:
table2 = table2.round(4)
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$    $C^u$    $C^l$  \
T v    K                                                                        
1 0.50 75             27.4591     27.3116            0.0054  27.5449  27.3232   
       90             14.0228     13.5774            0.0328  14.4322  13.3746   
       100             6.5478      5.9578            0.0990   7.3757   5.2373   
       110             1.9861      1.2027            0.6513   2.6928   0.8673   
       125             0.2383      0.0060           38.5228   0.3888   0.0000   
  0.75 75             27.6453     27.4386            0.0075  27.7540  27.4721   
       90             14.4322     13.9589            0.0339  14.8389  13.7845   
       100             7.0165      6.4220            0.0926   7.7711   5.8148   
       110             2.2323      1.3957            0.5994   2.8860   1.1914   
       125             0.2268      0.0051           43.3222   0.3693   0.0000   
  0.90 75             27.7607     27.5319            0.0083  27.8794  27.5708   
       90             14.6447     14.1706            0.0335  15.0469  14.0015   
       100             7.2406      6.6586            0.0874   7.9595   6.0911   
       110             2.3458      1.4925            0.5717   2.9714   1.3453   
       125             0.2209      0.0048           44.6008   0.3591   0.0000   
2 0.50 75             29.9060     29.6091            0.0100  30.0965  29.5989   
       90             17.3836     16.6890            0.0416  18.0287  16.3435   
       100            10.3539      9.4982            0.0901  11.4680   8.5573   
       110             5.1992      4.2383            0.2267   6.5056   3.0927   
       125             1.5182      0.5537            1.7417   2.1768   0.4562   
  0.75 75             30.2213     29.8354            0.0129  30.4404  29.8647   
       90             17.9767     17.2421            0.0426  18.6099  16.9462   
       100            11.0685     10.1977            0.0854  12.1011   9.3881   
       110             5.8436      4.8520            0.2044   7.0529   3.8755   
       125             1.7891      0.7026            1.5464   2.4236   0.7564   
  0.90 75             30.4099     29.9823            0.0143  30.6407  30.0322   
       90             18.2880     17.5383            0.0427  18.9119  17.2668   
       100            11.4233     10.5484            0.0829  12.4153   9.7996   
       110             6.1567      5.1560            0.1941   7.3180   4.2558   
       125             1.9286      0.7811            1.4690   2.5470   0.9164   

             $Var^u$   $Var^l$  $Var[Sy]$   $z_y$  
T v    K                                           
1 0.50 75   141.9349   61.6949   110.8710  0.3871  
       90   141.9349   61.6949   110.8710  0.3871  
       100  141.9349   61.6949   110.8710  0.3871  
       110  141.9349   61.6949   110.8710  0.3871  
       125  141.9349   61.6949   110.8710  0.3871  
  0.75 75   137.2473   60.0173   107.4557  0.3858  
       90   137.2473   60.0173   107.4557  0.3858  
       100  137.2473   60.0173   107.4557  0.3858  
       110  137.2473   60.0173   107.4557  0.3858  
       125  137.2473   60.0173   107.4557  0.3858  
  0.90 75   134.6077   59.0700   105.5446  0.3847  
       90   134.6077   59.0700   105.5446  0.3847  
       100  134.6077   59.0700   105.5446  0.3847  
       110  134.6077   59.0700   105.5446  0.3847  
       125  134.6077   59.0700   105.5446  0.3847  
2 0.50 75   403.7162  176.9713   316.9234  0.3828  
       90   403.7162  176.9713   316.9234  0.3828  
       100  403.7162  176.9713   316.9234  0.3828  
       110  403.7162  176.9713   316.9234  0.3828  
       125  403.7162  176.9713   316.9234  0.3828  
  0.75 75   391.8035  172.5531   308.3564  0.3806  
       90   391.8035  172.5531   308.3564  0.3806  
       100  391.8035  172.5531   308.3564  0.3806  
       110  391.8035  172.5531   308.3564  0.3806  
       125  391.8035  172.5531   308.3564  0.3806  
  0.90 75   385

## figure 1

$$f_{\bar{S}}(K)=\mathrm{e}^{r T} \frac{\partial \bar{C}^2[K]}{\partial K^2}$$

In [55]:
from tqdm import tqdm
i = 0
K = 100
C_sim_list = list()
C_sim_result_list= list()
power_num = 5
sim_times = 10 ** power_num
rho = np.eye(3)
(T,v,K) = table2.index.values[i]

np.random.seed(20240321) 
Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=rho, size=sim_times)
Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)
omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
K_i = K / weight.copy() / len(table1)
X_price_vector = X_0 * np.exp((r - q + omega) * T) * np.exp(np.outer(mu, Gamma_values)).T * np.exp(sigma * np.sqrt(Gamma_values.reshape(-1, 1)) * Normal_values)
S_price_vector = np.dot(X_price_vector, weight)
C_sim_vector_list = (S_price_vector - K) * (S_price_vector - K > 0)
C_sim = sum(C_sim_vector_list) * np.exp(-r * T) / len(C_sim_vector_list) # 原论文是27.3230
C_sim_result_list.append(C_sim)
C_sim_list = list()

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [22]:
import torch

# 创建一个需要求导的张量
x = torch.tensor(2.0, requires_grad=True)
x_i = x / torch.tensor([2.0,2.0], requires_grad=True)

# 定义一个函数 y = x^2
y = sum(x_i ** 2 + x_i)

# 对 y 进行求导
y.backward()

# 打印导数值
print(x.grad)  # 输出：tensor(4.)


tensor(3.)


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [54]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

i = 0
K = 100
C_sim_list = list()
C_sim_result_list = list()
rho = np.eye(3)
power_num = 5
sim_times = 10 ** power_num
(T, v, K) = table2.index.values[i]

np.random.seed(20240321)
Normal_values = multivariate_normal.rvs(mean=[0, 0, 0], cov=rho, size=sim_times)
Gamma_values = np.random.gamma(shape=T / v, scale=v, size=sim_times)
omega = 1 / v * np.log(1 - 1 / 2 * (sigma ** 2) * v - mu * v)
K_i = K / weight.copy() / len(table1)
X_price_vector = X_0 * np.exp((r - q + omega) * T) * np.exp(np.outer(mu, Gamma_values)).T * np.exp(
    sigma * np.sqrt(Gamma_values.reshape(-1, 1)) * Normal_values)
S_price_vector = np.dot(X_price_vector, weight)
C_sim_vector_list = torch.tensor((S_price_vector - K) * (S_price_vector - K > 0))
C_sim = torch.sum(C_sim_vector_list) * torch.exp(-r * T) / len(C_sim_vector_list)  # 原论文是27.3230
C_sim_result_list.append(C_sim.item())

# 求导数
C_sim.backward()

# 输出导数值
grad_C_sim = C_sim_vector_list.grad

# 将梯度值转换为NumPy数组
grad_C_sim_np = grad_C_sim.numpy()

# 输出结果
print(grad_C_sim_np)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [23]:
from tqdm import tqdm
i = 0
K = 100
C_sim_list = list()
C_sim_result_list= list()
power_num = 5
sim_times = 10 ** power_num
(T,v,K) = table2.index.values[i]

np.random.seed(20240321) 
Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=rho, size=sim_times)
Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)
omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
K_i = K / weight.copy() / len(table1)
X_price_vector = X_0 * np.exp((r - q + omega) * T) * np.exp(np.outer(mu, Gamma_values)).T * np.exp(sigma * np.sqrt(Gamma_values.reshape(-1, 1)) * Normal_values)
S_price_vector = np.dot(X_price_vector, weight)
C_sim_vector_list = (S_price_vector - K) * (S_price_vector - K > 0)
C_sim = sum(C_sim_vector_list) * np.exp(-r * T) / len(C_sim_vector_list) # 原论文是27.3230
C_sim_result_list.append(C_sim)
C_sim_list = list()
C_sim.backward()

AttributeError: 'numpy.float64' object has no attribute 'backward'

In [37]:
rho = rho.clone().detach().requires_grad_(True)
v = v.clone().detach().requires_grad_(True)
mu = mu.clone().detach().requires_grad_(True)
sigma = sigma.clone().detach().requires_grad_(True)
omega = omega.clone().detach().requires_grad_(True)
T = T.clone().detach().requires_grad_(True)
Gamma_values = Gamma_values.clone().detach().requires_grad_(True)
Normal_values = Normal_values.clone().detach().requires_grad_(True)

In [49]:
from tqdm import tqdm
i = 0
K = torch.tensor(100.0, requires_grad=True)
C_sim_list = list()
C_sim_result_list= list()
power_num = 5
sim_times = 10 ** power_num
(T,v,K) = table2.index.values[i]
np.random.seed(20240321) 
Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=np.array(rho), size=np.array(sim_times))
Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
K_i = K / weight.copy() / len(table1)
X_price_vector = X_0 * np.exp((r - q + omega) * T) * np.exp(np.outer(mu, Gamma_values)).T * np.exp(sigma * np.sqrt(Gamma_values.reshape(-1, 1)) * Normal_values)
S_price_vector = np.dot(X_price_vector, weight)
C_sim_vector_list = (S_price_vector - K) * (S_price_vector - K > 0)
C_sim = sum(C_sim_vector_list) * np.exp(-r * T) / len(C_sim_vector_list) # 原论文是27.3230
C_sim_result_list.append(C_sim)
C_sim_list = list()

In [74]:
C_sim.backward()


AttributeError: 'numpy.float64' object has no attribute 'backward'